In [20]:
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.dialects import postgresql

In [55]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import relationship, Session

Base = declarative_base()


class WasstraatModel(Base):
    __abstract__ = True

    herkomst = Column(Text)
    key = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)

class ABR(Base):
    __tablename__ = 'Def_ABR'
    primary_key = Column(Integer, primary_key=True, autoincrement=True)
    concept = Column(String(256))
    code = Column(String(80))
    label = Column(String(256))
    note = Column(Text)
    beginjaar = Column(Integer)
    eindjaar = Column(Integer)
    bron = Column(String(256))
    opmerkingen = Column(Text)
    uri = Column(String(256))
    uris = Column(Text)
    parentID = Column(ForeignKey('Def_ABR.primary_key', deferrable=True), index=True)
    parent = relationship('ABR')

    def __repr__(self):
        return f"({self.code}) " if self.code else "" + self.concept


engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
sess = Session(engine)

topq = sess.query(ABR.primary_key)
topq = topq.filter(ABR.primary_key == 31)
topq = topq.cte('cte', recursive=True)

bottomq = sess.query(ABR.primary_key)
bottomq = bottomq.join(topq, ABR.parentID == topq.c.primary_key)

recursive_q = topq.union(bottomq)
q = sess.query(recursive_q)

df = pd.read_sql(q.statement, sess.bind)

#str(q.statement.compile(dialect=postgresql.dialect()))
#pd.read_sql(topq.statement, sess.bind)

# .outerjoin(comments_subq, Article.id == comments_subq.c.article_id)

query = sess.query(ABR).filter(ABR.primary_key.in_(q))
df = pd.read_sql(query.statement, sess.bind)
df

,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,uri,uris,parentID
0,31,materialen,None,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/2...,{https://data.cultureelerfgoed.nl/term/id/abr/...,30
1,32,vervallen materialen,None,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/v...,{https://data.cultureelerfgoed.nl/term/id/abr/...,31
2,33,magering,KMA,None,Alle aan de natuurlijke klei toegevoegde mater...,None,None,None,was ABR specifiek niet in KNA.,https://data.cultureelerfgoed.nl/term/id/abr/3...,{https://data.cultureelerfgoed.nl/term/id/abr/...,32
3,34,calciet,KMC,None,Calciet is een mineraal dat aan klei kan worde...,None,None,None,vervallen,https://data.cultureelerfgoed.nl/term/id/abr/1...,{https://data.cultureelerfgoed.nl/term/id/abr/...,33
4,35,organische magering,KMO,plantaardige magering,Plantaardige magering bestaat uit alle planten...,None,None,None,vervallen,https://data.cultureelerfgoed.nl/term/id/abr/b...,{https://data.cultureelerfgoed.nl/term/id/abr/...,33
...,...,...,...,...,...,...,...,...,...,...,...,...
113,145,Noordelijk,SVN,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/e...,{https://data.cultureelerfgoed.nl/term/id/abr/...,128
114,146,Obourg,None,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/c...,{https://data.cultureelerfgoed.nl/term/id/abr/...,128
115,147,Rullen,None,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/6...,{https://data.cultureelerfgoed.nl/term/id/abr/...,128
116,148,gemengd,GEM,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/6...,{https://data.cultureelerfgoed.nl/term/id/abr/...,31


In [32]:
"""
for instance in session.query(User).order_by(User.id):
...     print(instance.name, instance.fullname)
"""

q = sess.query(bottomq)

for row in q:
    print("ID: ", row["primary_key"], "parentID: ", row["parentID"], "concept:", row["concept"], )


ArgumentError: Column expression or FROM clause expected, got <sqlalchemy.sql.selectable.Select object at 0xffff37b235e0> object resolved from <sqlalchemy.orm.query.Query object at 0xffff37af9180> object. To create a FROM clause from a <class 'sqlalchemy.sql.selectable.Select'> object, use the .subquery() method. (Background on this error at: https://sqlalche.me/e/14/89ve)